In [1]:
# Set project root
import os
import sys

# Manually set the path to the project root
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

In [2]:
import matplotlib.pyplot as plt
import numpy as np

from src.geometry import (
    compute_distance_from_homography,
    derive_metric_homography,
)
from src.matching import template_match
from src.measurement_data import load_measurements_from_yaml
from src.utils import load_calibration_json

In [3]:
# Load measurement data
data = load_measurements_from_yaml("../assets/measurements.yaml")

# Load camera calibration
K, dist_coeffs, image_size = load_calibration_json("../assets/camera_calibration.json")

scenes = data.get_all_scenes()

In [4]:
def invert_K(K):
    """Invert the camera intrinsic matrix K."""
    fx, fy = K[0, 0], K[1, 1]
    cx, cy = K[0, 2], K[1, 2]
    s =  K[0, 1]

    return np.array(
        [
            [1 / fx, -s / (fx * fy), (s * cy - cx * fy) / (fx * fy)],
            [0, 1 / fy, -cy / fy],
            [0, 0, 1]
        ]
    )

In [5]:
K_inv = invert_K(K)

# Check that the inverse is correct
assert np.allclose(np.dot(K, K_inv), np.eye(3)), "Inverse of K is not correct"

In [6]:
K

array([[894.67311331,   0.        , 485.2289589 ],
       [  0.        , 896.1589482 , 633.03618469],
       [  0.        ,   0.        ,   1.        ]])

In [7]:
# Homography for template T4
homography_T4 = np.array([
    [-0.17847302636271037, 0.23154580670332958, 318.610165400254],
    [-0.22057531122839966, -0.1693379863214316, 884.8510156631432],
    [4.816726125456617e-06, 1.5910372937632323e-05, 1.0]
])

# Homography for template T2
homography_T2 = np.array([
    [0.4350728790206812, 0.28880647750979427, 562.1676602052514],
    [-0.2177196474457285, 0.5507672329687451, 387.02951199786924],
    [-2.7309055511935735e-05, 0.00020572633668148338, 1.0]
])

# Homography for template T3
homography_T3 = np.array([
    [0.3039986433310253, -0.09182963405831032, 334.3373396385607],
    [0.11501613666903568, 0.3284311476213451, 84.84437146194264],
    [-1.2305953521946654e-05, 6.321638948067824e-05, 1.0]
])

In [8]:
import numpy as np
from typing import List, Tuple
from scipy.linalg import svd, cholesky, eig, inv


def solve_B_from_homographies(Hs: List[np.ndarray]) -> np.ndarray:
    """
    Solve for the Dual Image of the Absolute Conic (DIAC) matrix B = K^{-T} K^{-1}
    given a list of 3 homographies mapping known planar template coordinates to image pixels.

    Each homography contributes two linear constraints on the symmetric B. We
    assemble these into a 6x6 system A b = 0, solve via SVD, and reassemble B.
    """
    A = []
    for H in Hs:
        h1, h2 = H[:,0], H[:,1]
        # Constraint: h1^T B h2 = 0
        A.append([
            h1[0]*h2[0],
            h1[0]*h2[1] + h1[1]*h2[0],
            h1[1]*h2[1],
            h1[0]*h2[2] + h1[2]*h2[0],
            h1[1]*h2[2] + h1[2]*h2[1],
            h1[2]*h2[2]
        ])
        # Constraint: h1^T B h1 = h2^T B h2
        A.append([
            h1[0]*h1[0] - h2[0]*h2[0],
            2*(h1[0]*h1[1] - h2[0]*h2[1]),
            h1[1]*h1[1] - h2[1]*h2[1],
            2*(h1[0]*h1[2] - h2[0]*h2[2]),
            2*(h1[1]*h1[2] - h2[1]*h2[2]),
            h1[2]*h1[2] - h2[2]*h2[2]
        ])
    A = np.array(A)

    # Solve A b = 0 by SVD; b is the last column of Vt
    _, _, Vt = svd(A)
    b = Vt[-1]
    # Form B and normalize so B[2,2] = 1 (avoid divide-by-zero)
    B = np.array([
        [b[0], b[1], b[3]],
        [b[1], b[2], b[4]],
        [b[3], b[4], b[5]]
    ], dtype=float)
    if np.isclose(B[2,2], 0):
        raise ValueError("Degenerate template homographies: B[2,2] ~ 0")
    B /= B[2,2]
    return B


def enforce_positive_definite(B: np.ndarray) -> np.ndarray:
    """
    Clamp the eigenvalues of symmetric B to be >= epsilon, returning a PD matrix.
    """
    eigvals, eigvecs = eig(B)
    eigvals = np.real(eigvals)
    eigvecs = np.real(eigvecs)
    eigvals_clamped = np.clip(eigvals, 1e-6, None)
    return eigvecs @ np.diag(eigvals_clamped) @ eigvecs.T


def calibrate_from_homographies(Hs: List[np.ndarray]) -> Tuple[np.ndarray, np.ndarray]:
    """
    Estimate K from planar homographies.

    1. Solve for B = K^{-T}K^{-1}
    2. Clamp B to PD
    3. Invert and Cholesky: if cholesky(KKT, lower=False)=R s.t. R^T R=KKT,
       then K = R is upper-triangular.
    """
    # 1) Build and normalize B
    B = solve_B_from_homographies(Hs)
    # 2) Ensure positive-definiteness
    B_pd = enforce_positive_definite(B)
    # 3) Invert to get K K^T
    KKT = inv(B_pd)
    # Cholesky upper-triangular R s.t. R^T R = KKT
    R = cholesky(KKT, lower=False)
    K = R  # R is the intrinsic matrix K
    return K, B_pd

In [9]:
# Calibrate camera intrinsics from the homographies
K_est, B_est = calibrate_from_homographies([homography_T4, homography_T2, homography_T3])
print("Estimated Camera Intrinsics K:\n", K_est)
print("Estimated DIAC B:\n", B_est)

Estimated Camera Intrinsics K:
 [[ 9.99999996e+02  3.06470536e-06 -8.67807874e-02]
 [ 0.00000000e+00  9.99999999e+02  3.53339334e-02]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]]
Estimated DIAC B:
 [[ 1.00753091e-06 -3.06630965e-09  8.67807878e-05]
 [-3.06630965e-09  1.00124849e-06 -3.53339335e-05]
 [ 8.67807878e-05 -3.53339335e-05  1.00000000e+00]]


In [10]:
# Difference from original K
K_diff = K - K_est
print("Difference from Original K:\n", K_diff)

Difference from Original K:
 [[-1.05326883e+02 -3.06470536e-06  4.85315740e+02]
 [ 0.00000000e+00 -1.03841051e+02  6.33000851e+02]
 [ 0.00000000e+00  0.00000000e+00  5.99520433e-15]]
